In [ ]:
import pandas as pd

LABELS = {
    '0xa69babef1ca67a37ffaf7a485dfff3382056e78c': 'CEX_SCP',
    '0x51c72848c68a965f66fa7a88855f9f7784502a7f': 'CEX_wintermute',
    '0x1bf621aa9cee3f6154881c25041bb39aed4ca7cc': 'CEX_a7cc',
    '0xe8c060f8052e07423f71d445277c61ac5138a2e5': 'MEV_a2e5',
    '0xbeb5fd030ffb0fbc95d68113c1c796eff65526d7': 'CEX_26d7',
    '0x6f1cdbbb4d53d226cf4b917bf768b94acbab6168': 'CEX_6168',
    '0x9def7cde171841a9f0724124ca0b01a622d749e4': 'CEX_49e4',
    '0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad': 'UniversalRouter',
    '0x6b75d8af000000e20b7a7ddf000ba900b4009a80': 'MEV_jaredfromsubway',
    '0x5f515f6c524b18ca30f7783fb58dd4be2e9904ec': '1inch',
    '0xe37e799d5077682fa0a244d46e5649f71457bd09': '1inch',
    '0xe592427a0aece92de3edee1f18e0157c05861564': 'SwapRouter',
    '0x1111111254eeb25477b68fb85ed929f73a960582': 'AggregationRouterV5',
    '0xdef1c0ded9bec7f1a1670819833240f027b25eff': 'ZeroEx',
}




In [ ]:
df_tx_bribe = pd.read_csv('/data/hackthon/swaps/uni_v3/2024-04-01_bribe.csv')
df_block_bribe = pd.read_csv('/home/hruan/PBS/bribe_2024-04-01_2024-04-02.csv')
df_block_bribe['bribe_p'] = df_block_bribe['bribe_p'].fillna(0)
df_block_bribe['total_bribe'] = df_block_bribe['bribe_b'] + df_block_bribe['bribe_p'] 
df_block_bribe


In [ ]:
count_builder = df_block_bribe.groupby('builder').size().reset_index()
count_builder = count_builder.rename(columns={0: 'count'})
count_builder = count_builder.sort_values(by='count', ascending=False)
count_builder['ratio'] = count_builder['count'] / count_builder['count'].sum()
count_builder = count_builder.reset_index(drop=True)
count_builder[count_builder['ratio'] > 0.01]

In [ ]:


def extra_data_decoded(x):
    try:
        decoded_text = x.decode('utf-8')
        return decoded_text
    except Exception as e:
        return x.hex()
    
from web3 import Web3
w3=Web3(Web3.HTTPProvider("http://127.0.0.1:8547"))

df_block_bribe=df_block_bribe[df_block_bribe["bribe_p"]>0]
df_block_bribe["bribe_p/total_bribe"]=df_block_bribe["bribe_p"]/df_block_bribe["total_bribe"]
count_builder=pd.DataFrame(df_block_bribe.groupby("builder")["bribe_p"].count().rename("cnt")).merge(df_block_bribe.groupby("builder")[["bribe_b","bribe_p","bribe_p/total_bribe"]].mean().rename(columns={"bribe_b":"mean bribe_b","bribe_p":"mean bribe_p","bribe_p/total_bribe":"mean bribe_p/total_bribe"}),right_index=True,left_index=True).merge(df_block_bribe.groupby("builder")["bribe_b"].sum().rename("sum bribe_b"),right_index=True,left_index=True).merge(df_block_bribe.groupby("builder")["block"].first().map(lambda x:extra_data_decoded(w3.eth.get_block(x)["extraData"])).rename("extraData"),right_index=True,left_index=True).sort_values(by="cnt",ascending=False)
count_builder["mean bribe_p/total_bribe"]=count_builder["mean bribe_p"]/(count_builder["mean bribe_b"]+count_builder["mean bribe_p"])
count_builder



In [ ]:
# > 1% builder
builder_labels = {
    '0x95222290DD7278Aa3Ddd389Cc1E1d165CC4BAfe5' : 'Beaver',
    '0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97' : 'Titan',
    '0x1f9090aaE28b8a3dCeaDf281B0F12828e676c326' : 'Rsync',
    '0x388C818CA8B9251b393131C08a736A67ccB19297' : 'Lido',
    '0xDAFEA492D9c6733ae3d56b7Ed1ADB60692c98Bc5' : 'Flashbots',
    '0x88c6C46EBf353A52Bdbab708c23D0c81dAA8134A' : 'Jetbldr',
    #'0xf15689636571dba322b48E9EC9bA6cFB3DF818e1' : 'Penguin',    
}

In [ ]:
builder = '0x95222290DD7278Aa3Ddd389Cc1E1d165CC4BAfe5'
block_index_for_builder = df_block_bribe[df_block_bribe['builder'] == builder]['block']
total_bribe_for_builder = df_block_bribe[df_block_bribe['builder'] == builder]['total_bribe'].sum()
total_bribe_for_builder



In [ ]:
df_tx_bribe_for_builder = df_tx_bribe[df_tx_bribe['block_num'].isin(block_index_for_builder)].copy()
df_tx_bribe_for_builder

In [ ]:
# Categorize by label, count bribe; labels not in LABELS are others
df_tx_bribe_for_builder['label'] = df_tx_bribe_for_builder['to_address'].apply(lambda x: LABELS.get(x, 'others'))
df_tx_bribe_for_builder = df_tx_bribe_for_builder.groupby('label').agg({'bribe': 'sum'})
df_tx_bribe_for_builder['ratio'] = df_tx_bribe_for_builder['bribe'] / total_bribe_for_builder
df_tx_bribe_for_builder['builder'] = builder_labels.get(builder, builder)
df_tx_bribe_for_builder = df_tx_bribe_for_builder.reset_index(drop=True)
df_tx_bribe_for_builder


In [ ]:
tmp = []
for builder in builder_labels.keys():
    block_index_for_builder = df_block_bribe[df_block_bribe['builder'] == builder]['block']
    total_bribe_for_builder = df_block_bribe[df_block_bribe['builder'] == builder]['total_bribe'].dropna().sum()
    df_tx_bribe_for_builder = df_tx_bribe[df_tx_bribe['block_num'].isin(block_index_for_builder)].copy()
    df_tx_bribe_for_builder['label'] = df_tx_bribe_for_builder['to_address'].apply(lambda x: LABELS.get(x, 'others'))
    df_tx_bribe_for_builder = df_tx_bribe_for_builder.groupby('label').agg({'bribe': 'sum'}).reset_index()
    df_tx_bribe_for_builder['ratio'] = df_tx_bribe_for_builder['bribe'] / total_bribe_for_builder
    df_tx_bribe_for_builder['builder'] = builder_labels.get(builder, builder)
    df_tx_bribe_for_builder = df_tx_bribe_for_builder.reset_index(drop=True)
    tmp.append(df_tx_bribe_for_builder)
bribe_proportion = pd.concat(tmp)
bribe_proportion

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(16, 9))
pivot_table = bribe_proportion.pivot_table(index='label', columns='builder', values='ratio')
pivot_table = pivot_table.fillna(0)

sns.heatmap(pivot_table, annot=True, fmt=".2%", cmap='YlGnBu')
plt.xlabel('builder')
plt.ylabel('label')
plt.title('Bribe proportion for different builder')
plt.tight_layout()
# plt.show()
plt.savefig('img/heatmap_for_builder.png', dpi=300)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 8))
ax = sns.barplot(x="ratio", y="builder", hue="label", data=bribe_proportion, palette="muted")


plt.xlabel('Bribe proportion')
plt.ylabel('Builder')

plt.title('Bribe proportion for different builder')
plt.legend(title='Label')
plt.tight_layout()
# plt.show()
plt.savefig('img/bar_for_builder.png', dpi=300)

In [ ]:
cex_bot = '0xa69babef1ca67a37ffaf7a485dfff3382056e78c'
df_tx_bribe_for_bot = df_tx_bribe[df_tx_bribe['to_address'] == cex_bot].copy()
df_tx_bribe_for_bot['builder'] = df_tx_bribe_for_bot['block_num'].apply(lambda x: df_block_bribe[df_block_bribe['block'] == x]['builder'].values[0])
df_tx_bribe_for_bot = df_tx_bribe_for_bot.groupby('builder').agg({'tx_hash': 'count'}).reset_index()
df_tx_bribe_for_bot = df_tx_bribe_for_bot.rename(columns={'tx_hash': 'count'})
df_tx_bribe_for_bot

In [ ]:
bribe_tx_count = []
for cex_bot in LABELS.keys():
    df_tx_bribe_for_bot = df_tx_bribe[df_tx_bribe['to_address'] == cex_bot].copy()
    df_tx_bribe_for_bot['builder'] = df_tx_bribe_for_bot['block_num'].apply(lambda x: df_block_bribe[df_block_bribe['block'] == x]['builder'].values[0])
    df_tx_bribe_for_bot['builder'] = df_tx_bribe_for_bot['builder'].apply(lambda x: builder_labels.get(x, 'others'))
    df_tx_bribe_for_bot = df_tx_bribe_for_bot.groupby('builder').agg({'tx_hash': 'count', 'bribe' : sum}).reset_index()
    df_tx_bribe_for_bot = df_tx_bribe_for_bot.rename(columns={'tx_hash': 'count'})
    df_tx_bribe_for_bot['count'] = df_tx_bribe_for_bot['count'] / df_tx_bribe_for_bot['count'].sum()
    df_tx_bribe_for_bot['bribe_ratio'] = df_tx_bribe_for_bot['bribe'] / df_tx_bribe_for_bot['bribe'].sum()
    #print(df_tx_bribe_for_bot)
    #df_tx_bribe_for_bot = df_tx_bribe_for_bot.T
    df_tx_bribe_for_bot['label'] = LABELS[cex_bot]
    df_tx_bribe_for_bot = df_tx_bribe_for_bot.reset_index(drop=True)
    #print(df_tx_bribe_for_bot.transpose())
    bribe_tx_count.append(df_tx_bribe_for_bot)
bribe_tx_count = pd.concat(bribe_tx_count).reset_index(drop=True)
bribe_tx_count

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(16, 9))
pivot_table = bribe_tx_count.pivot_table(index='label', columns='builder', values='count')
pivot_table = pivot_table.fillna(0)

sns.heatmap(pivot_table, annot=True, fmt=".2%", cmap='YlGnBu')
plt.xlabel('Builder')
plt.ylabel('Label')
plt.title('Bribe Heatmap for different builder(Tx count)')
plt.tight_layout()
# plt.show()
plt.savefig('img/heatmap_tx_count.png', dpi=300)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(16, 9))
pivot_table = bribe_tx_count.pivot_table(index='label', columns='builder', values='bribe_ratio')
pivot_table = pivot_table.fillna(0)

sns.heatmap(pivot_table, annot=True, fmt=".2%", cmap='YlGnBu')
plt.xlabel('Builder')
plt.ylabel('Label')
plt.title('Bribe Heatmap for different builder(Bribe)')
plt.tight_layout()
# plt.show()
plt.savefig('img/heatmap_bribe.png', dpi=300)
